In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'C:\\Users\\omris\\OneDrive\\python\\me_toolbox'

# Helical Compression 

<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Helical_Compression_Spring.png" width=300"/> </td>
<td>
<img src="./img/Helical_Compression_Spring/Helical_Compression_Spring_end_type.png" width=400"/> </td>
</tr></table>

Importing the relevant Classes

In [2]:
from me_toolbox.springs import HelicalCompressionSpring, Spring

## Geometric properties

In [3]:
d = 6
D = 50

## Material properties

Table 10-4 from Shigley:
<table><tr>
<td>
<img src="./img/Helical_Compression_Spring/Tensile_strength_estimation.png" width=600"/> </td>
</tr></table>

Material options for use in Spring.material_prop method are:<br>
  * music wire<br>
  * oq&t wire<br>
  * hard-drawn wire<br>
  * chrome-vanadium wire<br>
  * chrome-silicon wire<br>
  * 302 stainless wire<br>
  * phosphore-bronze wire<br>

In [4]:
# music wire
G = 75e3  # [MPa]
E = 205e3  # [MPa]
Sut = Spring.material_prop('music wire', d, metric=True, verbose=False)
print(f"Sut = {Sut:.2f}")

Sut = 1705.13


Table 10-6 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Compression_Spring/present_of_tensile_strength.png" width=600"/> </td>
</tr></table>


In [5]:
yield_percent = 0.45

## Forces and spring rate

In [6]:
k = 6  # [N/mm]
Fmax = 500  # [N]
Fmin = 100  # [N]

## Defining the spring

Required parameters:
  * max_force - The maximum force in $[N]$
  * wire_diameter - The wire diameter in $[mm]$
  * spring_diameter - The spring diameter in $[mm]$
  * ultimate_tensile_strength - The ultimate tensile strength in $[MPa]$
  * shear_yield_precent - the precentage of the shear yield strength(Ssy) from the ultimate tensile strength (Sut)
  * shear_modulus - The material shear modulus in $[MPa]$
  * elastic_modulus - The material elastic modulus in $[MPa]$
  * end_type - The shape of the ends of the spring (used to calculate number of coils)
  * spring_rate - The spring rate in $[N/mm]$

Optional parameters: <br>
(the bold parameters are the defaults)

  * set_removed=**False** - If the spring's set has been removed (brought to solid state)
  * shot_peened=**False** - If the spring underwent shot peening
  * density=**None** - The Material density in $[kg/mm^3]$
  * working_frequency=**None** - The working frequency of the spring in $[Hz]$
  * anchors=**None** - The way the spring is held in both ends (used to check for buckling)
  * zeta=**0.15** - The overrun safety factor $(F_{solid}=(1+\zeta) \cdot F_{max})$


In [7]:
spring = HelicalCompressionSpring(max_force=Fmax,
                                  wire_diameter=d,
                                  spring_diameter=D,
                                  ultimate_tensile_strength=Sut,
                                  shear_yield_percent=yield_percent,
                                  shear_modulus=G,
                                  elastic_modulus=E,
                                  end_type='squared and ground',
                                  spring_rate=k,
                                  set_removed=False,
                                  shot_peened=True,
                                  density=None,
                                  working_frequency=None,
                                  anchors='fixed-hinged',
                                  zeta=0.25)

Note: active_coils=16.08 is not in range [3,15],this can cause non linear behavior
Note: buckling is accruing, max free length(free_length)= 190.14, free_length= 212.67


In [8]:
spring.solid_length

108.50516282764099

### Calculated geometric properties

In [9]:
print(f"Free Length (L0) = {spring.free_length:.2f}\n"
      f"Active Coils (Na) = {spring.active_coils:.2f}\n"
      f"Total Coils (Nt) = {spring.total_coils:.2f}")

Free Length (L0) = 212.67
Active Coils (Na) = 16.08
Total Coils (Nt) = 18.08


### Calculated material properties

In [10]:
print(f"Shear yield strength (Ssy) = {spring.shear_yield_strength:.2f}\n"
      f"Shear ultimate strength (Ssu) = {spring.shear_ultimate_strength:.2f}\n")

Shear yield strength (Ssy) = 767.31
Shear ultimate strength (Ssu) = 1142.44



## Static analysis

In [11]:
print(f"static safety factor = {spring.static_safety_factor():.2f}")
print(f"solid safety factor = {spring.static_safety_factor(solid=True):.2f}")

static safety factor = 2.21
solid safety factor = 1.77


## Fatigue analysis

In [12]:
nf, ns = spring.fatigue_analysis(Fmax, Fmin, 99.999, verbose=True)
print(f"Fatigue safety factor (nf) = {nf:.2f}\n"
      f"Static safety factor (ns) = {ns:.2f}")


Alternating force = 200.00, Mean force = 300.00
Alternating shear stress = 138.65, Mean shear stress = 207.98
Sse = 335.61
Fatigue safety factor (nf) = 1.68
Static safety factor (ns) = 2.21


## Buckling analysis

Buckling test, Table 10-2 from Shigley:
<table><tr>
<td> <img src="./img/Helical_Compression_Spring/Buckling_modes.png" width=400"/> </td>
<td> <img src="./img/Helical_Compression_Spring/Buckling_modes_table.png" width=500"/> </td>
</tr></table>


In [13]:
spring.buckling('fixed-hinged', verbose=True);

Buckling is accruing, the max safe length = 190.14,  but the free_length = 212.67


## Frequency analysis

In [14]:
spring.density = 7850  # [kg/m^3]
spring.working_frequency = 0.5  # [Hz]

The natural frequency must be greater than 20 times the working frequency<br>
$(f_{natural} = 20 \cdot f_{working})$

In [15]:
print(f"Natural frequency = {spring.natural_frequency:.2f}\n"
      f"20*Working frequency = {20*spring.working_frequency:.2f}")


Natural frequency = 2.60
20*Working frequency = 10.00
